In [0]:
#Description: CellLineNetsv2-MBN3 Performance on Breast Cancer cell line classification
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from mobilenetv3 import mobilenetv3

In [0]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = './drive/My Drive/images2'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))
print (classes)
print (inputs.size())
# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

#imshow(out, title=[class_names[x] for x in classes])
print (class_names)

tensor([2, 3, 2, 1])
torch.Size([4, 3, 224, 224])
['HC11', 'MCF10A', 'MCF12A', 'MCF7', 'MDA-MB-468']


In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
model = mobilenetv3()
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
set_parameter_requires_grad(model, True)

model.classifier[1] = nn.Linear(1280,5, bias=True)
model = model.to(device)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                        num_epochs=24)

Epoch 0/23
----------
train Loss: 1.6060 Acc: 0.2230
val Loss: 1.7107 Acc: 0.1901

Epoch 1/23
----------
train Loss: 1.6029 Acc: 0.2021
val Loss: 1.8442 Acc: 0.1901

Epoch 2/23
----------
train Loss: 1.5937 Acc: 0.2613
val Loss: 1.9068 Acc: 0.1736

Epoch 3/23
----------
train Loss: 1.5726 Acc: 0.2927
val Loss: 2.0553 Acc: 0.1983

Epoch 4/23
----------
train Loss: 1.5816 Acc: 0.2927
val Loss: 1.8739 Acc: 0.2397

Epoch 5/23
----------
train Loss: 1.5767 Acc: 0.2787
val Loss: 2.1576 Acc: 0.1983

Epoch 6/23
----------
train Loss: 1.5787 Acc: 0.2927
val Loss: 2.2800 Acc: 0.2231

Epoch 7/23
----------
train Loss: 1.6018 Acc: 0.2718
val Loss: 2.2553 Acc: 0.2066

Epoch 8/23
----------
train Loss: 1.5632 Acc: 0.3136
val Loss: 2.1044 Acc: 0.1983

Epoch 9/23
----------
train Loss: 1.5816 Acc: 0.2753
val Loss: 2.1130 Acc: 0.2479

Epoch 10/23
----------
train Loss: 1.6064 Acc: 0.2474
val Loss: 2.1357 Acc: 0.2149

Epoch 11/23
----------
train Loss: 1.5936 Acc: 0.2578
val Loss: 1.9828 Acc: 0.2066

Ep